## Github 연동하기

## 라이브러리 설치

In [ ]:
# !pip install schedule

## 라이브러리 불러오기

In [ ]:
# API 호출
# ## https://data.go.kr/tcs/dss/selectApiDataDetailView.do?publicDataPk=15057440
from urllib.request import Request, urlopen
# Request https://docs.python.org/ko/3/library/urllib.request.html#urllib.request.Request
# urlopen https://docs.python.org/ko/3/library/urllib.request.html#urllib.request.urlopen
from urllib.parse import urlencode, quote_plus
# urlenocde https://docs.python.org/ko/3/library/urllib.parse.html#urllib.parse.urlencode
# quote_plus https://docs.python.org/ko/3/library/urllib.parse.html#urllib.parse.quote_plus
from urllib.parse import unquote
import requests

# 시간 생성
import time
import datetime
from pytz import timezone, utc

# 폴더 자동 생성
import os

# 코드 스케줄링
import threading
import schedule

# XML 파싱
import xml.etree.ElementTree as ET

# 입력데이터 데이터프레임 작성
import pandas as pd
import numpy as np

# DataFrame 화면 출력                                                # DataFrame 화면 출력 설정이 변경되어 있습니다 #
from IPython.display import display
pd.options.display.max_columns = None
pd.options.display.max_rows = None # default = 60

## KEY unquote

In [ ]:
KEY = 'yEaR%2F3MDedRSlVJL%2F2pxnVg0yre1N5VF3RZ%2FUAt56MJ7J2mNpfqhUvy05pXV0uhHTVY7DbyCR8xmMaDdYga67Q%3D%3D' # 종현
# apiKEY = requests.utils.unquote(KEY)
apiKey = unquote(KEY)
# print(apiKey)
KEYSW = 'M%2B4%2FqUiadT8X8PhgFjaQLDu%2BIOgPMURfGsOX%2FmVxwHQVJgnVR%2FMPjDYXkuQNwUFbZXlfnX5Lls3SUCiCLIFjgQ%3D%3D'
apiKeySW = unquote(KEYSW)

## URL 생성

### ws.bus.go.kr url을 사용하는 경우 
1.   'ServiceKey' 대문자 S
2.   'busRouteId' 노선ID는 busRouteId

In [ ]:
# getArrInfoByRouteAll
url = 'http://ws.bus.go.kr/api/rest/arrive/getArrInfoByRouteAll'
queryParams = '?' + urlencode({ quote_plus('ServiceKey') : apiKey, quote_plus('busRouteId') : '204000046' })

In [ ]:
# getBusPosByRtid
# 노선ID로 차량들의 위치정보를 조회합니다
url = 'http://ws.bus.go.kr/api/rest/buspos/getBusPosByRtid'
queryParams = '?' + urlencode({ quote_plus('ServiceKey') : apiKey, quote_plus('busRouteId') : '204000046' })

### openapi url을 사용하는 경우 
1.   'serviceKey' 소문자 s
2.   'routeId' 노선ID는 routeId

In [ ]:
# 노선번호목록조회 busrouteservice
# 해당 노선번호의 노선ID, 노선유형, 운행지역 등을 제공합니다.
url = 'http://openapi.gbis.go.kr/ws/rest/busrouteservice' 
queryParams = '?' + urlencode({ quote_plus('serviceKey') : apiKey, quote_plus('keyword') : '7770' })

In [1]:
# 노선정보항목조회 busrouteservice/info
# 해당 노선에 대한 노선번호, 기점/종점 정류소, 첫차/막차시간, 배차간격, 운행업체 등의 운행계획 정보를 제공합니다.
url = 'http://openapi.gbis.go.kr/ws/rest/busrouteservice/info'
queryParams = '?' + urlencode({ quote_plus('serviceKey') : apiKey, quote_plus('routeId') : '204000046' })

NameError: name 'urlencode' is not defined

In [ ]:
# 경유정류소목록조회 busrouteservice/info
# 해당 노선이 정차하는 경유정류소 목록과 정류소명, 중앙차로여부, 회차점, 좌표값 등을 제공합니다.
url = 'http://openapi.gbis.go.kr/ws/rest/busrouteservice/station'
queryParams = '?' + urlencode({ quote_plus('serviceKey') : apiKey, quote_plus('routeId') : '204000046' })

In [ ]:
# 경유정류소목록조회 buslocationservice
# 해당 노선이 정차하는 경유정류소 목록과 정류소명, 중앙차로여부, 회차점, 좌표값 등을 제공합니다.
url = 'http://openapi.gbis.go.kr/ws/rest/buslocationservice'
queryParams = '?' + urlencode({ quote_plus('serviceKey') : apiKey, quote_plus('routeId') : '204000046' })

In [ ]:
url+queryParams

## @routeIdList 작성 선행

In [ ]:
routeIdList = \
[200000193, 
 204000059, 204000065, 232000072, 232000081, 229000072,
 222000088, 200000262, 234001511, 218000011, 229000083, 200000109,
 227000039, 233000140, 232000007, 234000013, 233000270, 219000027,
 232000098, 204000056, 234001290, 236000050, 234001138, 234000065,
 228000393, 234000027, 232000090, 234001251, 232000073, 228000395,
 232000005, 233000135, 233000125, 234001244, 216000026, 233000266,
 234000042, 216000043, 216000061, 234000130, 213000019, 234001694,
 233000258, 234000075, 234000884, 227000038, 228000184, 232000047,
 200000149, 200000145, 213000015, 228000181, 233000131, 228000262,
 234000310, 224000047, 234001271, 200000145, 224000050, 224000019,
 222000074, 204000024, 236000176, 229000111, 228000263, 222000078,
 234000882, 235000085, 234000069, 228000177, 234000079, 221000003,
 229000112, 227000040, 234000873, 228000388, 228000176, 204000070,
 234000051, 234000136, 222000073, 233000136, 234000873, 234001163,
 234000886, 234000015, 228000174, 229000023, 234000873, 204000046,
 234000148, 234000002, 229000028, 229000028, 222000076, 222000075,
 205000002, 233000265, 233000139, 227000019, 200000175, 228000175,
 228000182, 200000108, 207000070, 234000309, 234000313, 200000112,
 200000205, 222000107, 234000016, 200000110, 204000041, 228000389,
 234001204, 234001516, 200000104, 222000049, 236000149, 200000119,
 204000082, 221000033, 228000394, 234000031, 229000061, 222000169,
 234000050, 204000057, 234000324, 234000878, 235000092, 200000120,
 213000024, 234000011, 200000115, 229000097, 218000007, 233000032,
 232000103, 222000090, 218000010, 206000007, 204000081, 232000092,
 232000092, 219000016, 219000006, 200000150, 222000084, 222000046,
 233000031, 219000013, 233000031, 229000247, 222000137, 233000142,
 234001609, 234001203, 233000269
]

In [ ]:
seoulBusRouteIdList = [100100124, 100100001, 100100002, 100100003, 100100549, 100100006, 100100129, 100100130, 100100007, 100100131, 100100008, 100100009, 100100010, 100100011, 100100012, 100100013, 100100014, 100100016, 100100015, 100100132, 100100133, 100100134, 100100566, 107000001, 100100137, 100100138, 100100425, 100100139, 100100142, 100100143, 100100144, 100100145, 100100146, 100100147, 100100148, 100100149, 100100150, 100100151, 100100152, 100100153, 100100154, 100100155, 100100156, 100100157, 100100158, 100100159, 100100164, 100100165, 100100166, 100100170, 100100171, 100100172, 108000002, 100100017, 100100583, 100100175, 100100177, 100100178, 100100179, 100100181, 100100183, 100100184, 100100018, 100100019, 100100020, 100100021, 100100022, 100100023, 100100024, 100100025, 100100026, 100100027, 100100029, 100100030, 100100031, 100100032, 100100033, 100100034, 100100036, 100100185, 100100037, 110000002, 100100038, 100100186, 100100188, 100100189, 100100190, 100100522, 100100039, 100100191, 100100192, 100100193, 100100598, 100100194, 100100198, 100100199, 100100201, 100100202, 100100203, 100100204, 100100205, 100100206, 100100599, 100100611, 100100407, 100100595, 100100209, 100100210, 100100211, 104000006, 100100042, 100100043, 100100044, 100100046, 100100047, 100100048, 100100049, 100100051, 100100438, 100100579, 100100052, 100100053, 100100564, 100100212, 100100213, 100100215, 100100216, 100100218, 100100219, 100100220, 100100221, 100100222, 100100478, 100100498, 100100501, 100100578, 100100613, 100100496, 100100055, 100100056, 100100223, 100100224, 100100225, 100100226, 100100228, 100100229, 124000038, 100100232, 100100609, 100100612, 107000002, 100100553, 100100057, 124000036, 100100061, 100100596, 100100062, 100100063, 100100597, 100100064, 100100068, 100100409, 100100604, 100100234, 100100237, 100100427, 100100459, 100100070, 100100500, 100100454, 100100246, 100100247, 122000001, 113000002, 100100071, 100100605, 100100073, 100100075, 100100076, 100100077, 100100248, 100100410, 100100078, 100100079, 100100080, 100100081, 100100082, 100100083, 100100574, 100100249, 100100603, 100100250, 100100251, 100100252, 100100253, 100100254, 100100255, 100100257, 100100258, 100100259, 100100260, 100100261, 100100263, 100100264, 100100265, 100100266, 100100267, 100100268, 100100269, 100100272, 100100273, 100100274, 100100275, 100100276, 100100277, 100100278, 100100279, 100100280, 100100281, 100100282, 100100283, 100100284, 100100285, 100100084, 100100286, 100100287, 100100288, 100100085, 100100411, 100100412, 100100413, 100100414, 100100415, 100100416, 100100506, 123000006, 100100417, 100100507, 123000007, 100100086, 100100584, 100100418, 100100419, 100100420, 100100421, 100100422, 100100423, 100100533, 100100538, 115000006, 100100087, 100100509, 123000008, 115000001, 100100088, 100100517, 100100089, 100100090, 100100091, 100100513, 100100373, 122000002, 124000013, 124000006, 100100289, 100100508, 123000009, 100100093, 100100094, 100100290, 100100096, 100100097, 124000009, 100100098, 100100291, 100100292, 100100293, 100100294, 100100521, 100100576, 100100099, 100100497, 115000007, 123000002, 100100102, 100100295, 100100453, 100100297, 116000002, 100100298, 100100299, 100100550, 100100300, 100100301, 100100302, 100100303, 100100304, 100100305, 100100306, 100100307, 100100308, 100100309, 100100311, 100100312, 100100313, 100100601, 100100602, 100100316, 100100318, 100100320, 115000005, 114000001, 100100322, 100100329, 100100436, 100100567, 100100573, 100100572, 100100570, 100100569, 100100568, 115000002, 100100330, 100100331, 100100332, 100100451, 100100495, 100100551, 111000008, 100100440, 100100103, 100100450, 100100449, 100100448, 100100447, 100100337, 100100338, 100100339, 100100340, 100100341, 100100342, 100100446, 100100104, 100100105, 100100107, 100100587, 100100511, 123000011, 100100110, 100100111, 100100112, 100100344, 100100499, 100100537, 123000010, 100100114, 100100565, 100100116, 100100117, 100100118, 111000014, 100100345, 100100346, 100100347, 100100437, 100100348, 100100349, 100100462, 124000008, 100100352, 100100353, 100100354, 100100357, 100100358, 100100359, 111000011, 100100360, 111000010, 100100363, 100100364, 100100552, 111000012, 106000001, 124000010, 103000001, 100100383, 108000001, 116000001, 113000001, 111000009, 100100525, 100100387, 112000001, 100100389, 100100390, 100100391, 100100392, 100100397, 100100398, 100100400, 100100402, 100100607, 100100406, 100100593, 100100610, 100100592, 100100586, 100100591, 100100585, 124000016, 124000015, 115000009, 100100589, 100100588, 115000008, 122900003, 122900006, 122900004, 122900002, 122900005, 122900007, 122900001, 122900008, 122900009, 124900002, 124900003, 124900001, 108900006, 108900002, 108900003, 108900005, 108900008, 108900007, 108900013, 108900004, 108900009, 108900001, 108900012, 115900006, 115900003, 115900004, 115900001, 115900005, 115900008, 115900002, 115900007, 120900005, 120900008, 120900003, 120900009, 120900010, 120900004, 120900006, 120900007, 120900002, 114900004, 104900005, 104900003, 104900002, 104900001, 104900004, 116900013, 116900014, 116900010, 116900011, 116900009, 116900012, 116900006, 116900015, 116900007, 116900003, 116900004, 116900002, 116900001, 116900005, 116900016, 117900008, 111900013, 117900006, 117900003, 117900002, 117900001, 117900004, 117900007, 117900005, 117900010, 110900004, 110900009, 110900005, 110900001, 110900003, 110900007, 110900006, 110900002, 110900008, 109900011, 109900010, 109900001, 108900010, 109900007, 109900008, 109900004, 109900003, 109900002, 109900006, 109900005, 105900003, 105900002, 105900006, 105900001, 119900007, 119900013, 119900020, 119900022, 119900023, 119900014, 119900018, 119900009, 119900019, 119900006, 119900011, 119900021, 120900001, 119900012, 119900010, 119900017, 119900016, 119900015, 119900008, 119900001, 119900024, 102900001, 113900005, 113900002, 113900011, 113900012, 113900010, 113900013, 113900015, 113900008, 113900004, 113900003, 113900007, 113900006, 113900014, 113900009, 113900001, 113900016, 112900010, 112900014, 112900015, 112900004, 112900011, 112900009, 112900007, 112900003, 100900012, 112900012, 112900013, 112900001, 112900006, 112900002, 112900005, 112900008, 121900009, 121900008, 121900005, 119900003, 119900002, 121900011, 121900002, 121900007, 121900003, 121900006, 121900013, 121900010, 121900012, 119900005, 119900004, 121900016, 121900015, 121900004, 121900001, 121900014, 103900003, 103900004, 103900006, 101900001, 103900007, 103900005, 103900002, 103900001, 103900008, 103900009, 103900010, 107900008, 107900003, 107900002, 107900005, 107900011, 107900012, 107900013, 107900007, 107900006, 107900014, 107900017, 107900004, 107900001, 107900010, 107900016, 107900015, 105900004, 105900005, 107900009, 114900001, 114900002, 114900003, 116900018, 116900008, 118900004, 118900005, 118900001, 118900006, 118900003, 118900002, 118900007, 116900017, 118900008, 118900009, 114900006, 114900005, 102900003, 102900002, 102900004, 101900002, 111900002, 111900003, 111900001, 111900004, 111900005, 111900007, 111900006, 111900011, 111900012, 111900009, 111900010, 100900006, 100900008, 100900010, 100900011, 100900001, 100900004, 100900005, 100900003, 100900007, 100900009, 100900002, 106900001, 106900002]
print('서울시 버스 노선의 총 수 : ', len(seoulBusRouteIdList))
print(seoulBusRouteIdList)

## @현재 시간 생성 함수

In [ ]:
def getNowTime():
    fileDateTime = utc.localize(datetime.datetime.utcnow()).astimezone(timezone('Asia/Seoul'))       # 자정이 넘어간 시간 파일 경로가 바뀌는 문제를 해결해야 합니다.
    return fileDateTime

In [ ]:
print(getNowTime())
print(type(getNowTime()))
print(getNowTime().isoformat()) # string
print(getNowTime().isoformat()[:10])
print(getNowTime().isoformat()[11:19])

## @filePathList 생성

In [ ]:
# 폴더 자동 생성 함수
# https://data-make.tistory.com/170
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)

In [ ]:
########################################################################
## 코드를 실행하는 장치에 따라 달라지는 경로 입니다. 수정해서 사용하세요. ##
########################################################################

fileDevicepath = 'C:/Users/jongh/OneDrive/School/석사/1-2/자료구조/프로젝트/5.DATA'
# fileDevicepath = 'C:/Users/Administrator/OneDrive/School/석사/1-2/자료구조/프로젝트/5.DATA'    # AWS 사용시 선택하는 경로

# 기능 구분 폴더 입니다. 실제 데이터 기록시 반드시 확인하세요.
# fileAPIName = 'buslocationservice'                                                            # AWS 사용시 선택하는 폴더 이름
fileAPIName = 'testFolder1'

# 루트 폴더 입니다.
fileRootPath = '/Master/dataAPI/'

In [ ]:
def makeFilePathList():
    
    global fileDevicepath, fileRootPath, fileAPIName
    filePathList = []

    for routeId in routeIdList:
        
        # 고정된 이름 및 변수에 의해 자동으로 생성 되는 이름 입니다. 변수를 확인하세요.
        fileRouteId = str(routeId)                        # 파일 이름에 routeId를 포함 시킬지 확인합니다.
        fileDateTime = getNowTime().isoformat()[2:10]      # 자정이 넘어간 시간 파일 경로가 바뀌는 문제를 해결해야 합니다.
        fileExtension = '.txt'

        #파일의 폴더 경로와 이름을 각각 출력합니다.
        folderPath = str(fileDevicepath + fileRootPath + fileAPIName + '/' + fileRouteId)
        # print(folderPath)
        fileName = fileRouteId + '_' + fileDateTime + fileExtension
        # print(fileName)

        # 파일의 최종 경로를 출력합니다.
        filePath = str(folderPath + '/' + fileName)

        # 폴더를 생성합니다.
        createFolder(folderPath)

        # 파일 경로 리스트에 파일경로를 추가합니다.
        filePathList.append(filePath)
        
    return filePathList

In [ ]:
filePathList = makeFilePathList()
print(len(filePathList))
print(filePathList[0])

## @로그파일을 생성한다

In [ ]:
def makeLogFile():
    global fileDevicepath, fileRootPath, fileAPIName
    logfileRootPath = fileDevicepath + fileRootPath + fileAPIName + '/_log'
    fileDateTime = getNowTime().isoformat()[2:10]
    fileName = '_log.txt'
    
    logPath = logfileRootPath + '/' + fileDateTime + fileName 
    # print(logPath)
    # print(logfileRootPath)
    
    # logPath 경로에 폴더를 생성합니다.
    createFolder(logfileRootPath)
    
    return logPath

logPath = makeLogFile()

In [ ]:
def newLogFile(logPath):
    logFile = open(logPath, 'w', encoding = 'utf-8', newline = '\n')
    logFile.write(getNowTime().isoformat())
    logFile.write("\n")
    
    # logFile = open(logPath, 'a', encoding = 'utf-8', newline = '\n')
    # return logFile

newLogFile(logPath)

## "ws.bus.go.kr" 도메인 API 호출
* 로그파일 출력 코드가 필요합니다.
* 예외처리 코드가 필요합니다.

In [ ]:
def wsAPICall(routeIdList, filePathList):
    ###########################################################  
    url = 'http://ws.bus.go.kr/api/rest/buspos/getBusPosByRtid'# <<<<<<<<<<<<< 도메인 확인
    ###########################################################

    pastTime = getNowTime()

    print(pastTime.isoformat(),' API [ buslocationservice ] 호출을 [ 시작 ] 하였습니다.')  

    for routeId, filePath in zip(routeIdList, filePathList):
        # filePath에 있는 .txt파일을 append 모드로 엽니다.
        textFile = open(filePath, 'a', encoding = 'utf-8', newline = '\n')

        # API를 호출하여 XML 형식으로 된 string 데이터를 변수 "oneLineXML" 에 저장합니다.
        queryParams = '?' + urlencode({ quote_plus('serviceKey') : apiKey, quote_plus('busRouteId') : routeId })
        request = Request(url + queryParams)
        request.get_method = lambda: 'GET'
        oneLineXML = urlopen(request).read().decode('utf8')

        # xtree는 "ws.bus.go.kr" 도메인 API 호출에서 "headerCd"에 상관없이 3개의 태그를 갖습니다.
        xtree = ET.fromstring(oneLineXML)

        # comMsgHeader : []
        comMsgHeader = xtree[0]     

        # msgHeader : [headerCd, headerMsg, itemCount]
        msgHeader = xtree[1]
        headerCd = int(msgHeader[0].text)
        if(headerCd != 0):
            textFile.write(getNowTime().isoformat())
            textFile.write(' ')
            textFile.write(msgHeader[1].text)
            textFile.write("\n")
            print(getNowTime(), routeId, end=' ')
            print(msgHeader[1].text)
            continue;

        # msgBody : [ 23개 태그 ]
        msgBody = xtree[2]
        # itemTagList = ['busType', 'congetion', 'dataTm', 'fullSectDist', 'gpsX', 'gpsY', 'isFullFlag', 'islastyn', 'isrunyn', 'lastStTm', 'lastStnId', 'nextStId', 'nextStTm', 'plainNo', 'posX', 'posY', 'rtDist', 'sectDist', 'sectOrd', 'sectionId', 'stopFlag', 'trnstnid', 'vehId']
        for itemList in msgBody:
            itemValueList = []
            # itemValueList.append(getNowTime().isoformat())    # "ws.bus.go.kr" 도메인은 API 호출시간 ( "dataTm" ) 제공
            for item in itemList:
                itemValueList.append(item.text)
                textFile.write(item.text)
                textFile.write(' ')
            textFile.write("\n")
            # print(str(itemValueList))

    currentTime = getNowTime()
    print(currentTime.isoformat(),' API [ buslocationservice ] 호출을 [ 완료 ] 하였습니다.\t 소요시간: ', currentTime - pastTime)   

    textFile.close()
    
wsAPICall(routeIdList, filePathList)

## "openapi.gbis.go.kr" 도메인 API 호출

In [ ]:
def openAPICall(routeIdList, filePathList):
    ############################################################  
    url = 'http://openapi.gbis.go.kr/ws/rest/buslocationservice'# <<<<<<<<<<<<< 도메인 확인
    ############################################################

    pastTime = getNowTime()
    logFile = open(logPath, 'a', encoding = 'utf-8', newline = '\n')
    
    print(pastTime.isoformat(),'\t API [ buslocationservice ] 호출을 [ 시작 ] 하였습니다.')  

    for routeId, filePath in zip(routeIdList, filePathList):
        # filePath에 있는 .txt파일을 append 모드로 엽니다.
        textFile = open(filePath, 'a', encoding = 'utf-8', newline = '\n')

        # API를 호출하여 XML 형식으로 된 string 데이터를 변수 "oneLineXML" 에 저장합니다.
        queryParams = '?' + urlencode({ quote_plus('serviceKey') : apiKey, quote_plus('routeId') : routeId })
        request = Request(url + queryParams)
        request.get_method = lambda: 'GET'
        oneLineXML = urlopen(request).read().decode('utf8')
        
        # xtree는 "ws.bus.go.kr" 도메인 API 호출에서 "headerCd"에 상관없이 3개의 태그를 갖습니다.
        # [comMsgHeader, msgHeader, msgBody]
        xtree = ET.fromstring(oneLineXML)

        # comMsgHeader : []
        # comMsgHeader = xtree[0]


        # msgHeader : [queryTime, resultCode, resultMessage]
        # Index Error : IE01
        try:
            msgHeader = xtree[1]
        except IndexError:
            # textFile Index Error IE01 출력
            textFile.write(getNowTime().isoformat())
            textFile.write(" Index Error EC01\n")
            # logFile Index Error IE01 출력
            logFile.write(getNowTime().isoformat())
            logFile.write(' ')
            logFile.write(str(routeId))
            logFile.write(" Index Error EC01\n")
            logFile.write(oneLineXML)
            logFile.write('\n')
            # 프롬프트 Index Error IE01 출력
            print(getNowTime(), routeId, end=' ')
            print("Index Error EC01")
            continue
        
        # API Call Failure : CF01 
        resultCode = int(msgHeader[1].text)
        # print(resultCode)
        resultMessage = msgHeader[2]
        if(resultCode != 0):
            # textFile API Call Failure 출력
            textFile.write(getNowTime().isoformat())
            textFile.write(" API Call Failure CF01 ")
            textFile.write(resultMessage.text)
            textFile.write("\n")
            # logFile API Call Failure 출력
            logFile.write(getNowTime().isoformat())
            logFile.write(' ')
            logFile.write(str(routeId))
            logFile.write(" API Call Failure CF01 ")
            logFile.write(resultMessage.text)
            logFile.write("\n")
            # 프롬프트 API Call Failure 출력
            # print(getNowTime(), routeId, end=' ')
            # print(" API Call Failure CF01", end=' ')
            # print(resultMessage.text)
            continue;

        # msgBody : dateTime + [ 8개 태그 ]
        # Index Error 02 : IE02
        try:
            msgBody = xtree[2]
        except IndexError:
            # textFile Index Error IE02 출력
            textFile.write(getNowTime().isoformat())
            textFile.write(" Index Error IE02\n")
            # logFile Index Error 01 출력
            logFile.write(getNowTime().isoformat())
            logFile.write(' ')
            logFile.write(str(routeId))
            logFile.write(" Index Error IE02\n")
            logFile.write(oneLineXML)
            logFile.write('\n')
            # 프롬프트 Index Error IE02 출력
            print(getNowTime(), routeId, end=' ')
            print("Index Error IE2")
            continue
        
        #itemTagList = ['endBus', 'lowPlate', 'plateNo', 'plateType', 'remainSeatCnt', 'routeId', 'stationId', 'stationSeq']
        for busLocationList in msgBody:
            busLocationValueList = []
            textFile.write(getNowTime().isoformat())
            textFile.write(' ')
            # itemValueList.append(getNowTime().isoformat())    # "ws.bus.go.kr" 도메인은 API 호출시간 ( "dataTm" ) 제공
            for busLocation in busLocationList:
                # busLocationValueList.append(busLocation.text)
                textFile.write(busLocation.text)
                textFile.write(' ')
            textFile.write("\n")
            # print(busLocationValueList)

    currentTime = getNowTime()
    print(currentTime.isoformat(),'\t API [ buslocationservice ] 호출을 [ 완료 ] 하였습니다. 소요시간: ', currentTime - pastTime)   

    textFile.close()
    logFile.close()
    
openAPICall(routeIdList, filePathList)

## @Schedule 모듈 사용을 위한 API 호출 생성 함수 생성

In [ ]:
def newFile(routeIdList, filePathList):
    for routeId, filePath in zip(routeIdList, filePathList):
        textFile = open(filePath, 'w', encoding = 'utf-8', newline = '\n')
        textFile.write(str(routeId))
        textFile.write(' ')
        textFile.write(getNowTime().isoformat())
        textFile.write("\n")

In [ ]:
######################################
# newFile(routeIdList, filePathList)##   <<<  데이터파일을 초기화 합니다. 주의해서 사용하세요!
# newLogFile(logPath)               ##   <<<  로그파일을 초기화 합니다. 주의해서 사용하세요!
######################################

## @최종코드

In [ ]:
schedule.clear()
schedule.every(7).seconds.do(lambda: openAPICall(routeIdList, filePathList))
print(getNowTime(), "\t프로세스를 시작합니다.")

while True:
    try:
        schedule.run_pending()
        time.sleep(1)
    except:
        # logFile Timeout Error TE01 출력
        logFile = open(logPath, 'a', encoding = 'utf-8', newline = '\n')
        logFile.write(getNowTime().isoformat())
        logFile.write(' ')
        logFile.write(" Timeout Error TE01\n")
        # 프롬프트 Timeout Error TE01 출력
        print(getNowTime(), end=' ')
        print("Timeout Error TE01")
        logFile.close()
        continue

## @데이터 확인하기

In [2]:
import pandas as pd
# DataFrame 화면 출력                                                # DataFrame 화면 출력 설정이 변경되어 있습니다 #
from IPython.display import display
pd.options.display.max_columns = None
pd.options.display.max_rows = None # default = 60

In [37]:
# routeId 별 정류소 도착시간을 나타내는 2차원 배열을 생성 할 수 있습니다
# 마지막 정류소 번호가 필요합니다 : 경유정류소목록조회 API 서비스를 이용하여 가져옵니다.
# 중간 또는 처음과 끝에 발생하는 결측치를 채워야 합니다.
outputPath = 'C:/Users/jongh/OneDrive/School/석사/1-2/자료구조/프로젝트/5.DATA/Master/dataAPI/buslocationservice/204000046/204000046_20-11-13.txt'
itemTagList = ['endBus', 'lowPlate', 'plateNo', 'plateType', 'remainSeatCnt', 'routeId', 'stationId', 'stationSeq', 'Nan']
# itemTagList = ['endBus', 'plateNo', 'plateType', 'remainSeatCnt', 'routeId', 'stationId', 'stationSeq', 'Nan']
# itemTagList = ['lowPlate', 'plateNo', 'plateType', 'remainSeatCnt', 'routeId', 'stationId', 'stationSeq', 'Nan']               # 수정전 DateTime자료에 사용 ; endBus값이 DateTime에 붙음
itemTagList.insert(0, 'DateTime')
_df = pd.read_csv(outputPath, sep=' ', skiprows=[0], names=itemTagList)
_df = _df.drop(["Nan"], axis=1)
_df = _df.sort_values(['plateNo'])
_df = _df.dropna()
_df = _df.drop(_df[_df["endBus"]=="API"].index)
# _df

In [38]:
# 차량 운행기록을 "plateNo"를 기준으로 구분합니다.
plateNoList = _df.groupby(['plateNo']).size().index

In [64]:
# 각 차량 번호를 기준으로 반복문을 수행합니다.
for plateNo in plateNoList:
    
    # 특정 차량번호와 같은 운행기록을 "DateTime을 기준으로 오름차순으로 정렬합니다."
    sameBusList = _df[_df['plateNo'] == plateNo].sort_values(['DateTime']).reset_index(drop=True)
#     print(sameBusList)
    
    
    # 버스는 뒤로가지 않으므로
    # "stationSeq"가 작아지는 인덱스 i의 리스트를 만듭니다.
    indexList = []
    for i in range(1,len(sameBusList)):
        if(sameBusList.iloc[i-1]['stationSeq'] > sameBusList.iloc[i]['stationSeq']):
            indexList.append(i)
            print((sameBusList.iloc[i-1]['stationSeq'] , sameBusList.iloc[i]['stationSeq']))
            
    indexList.append(len(sameBusList))
    print(indexList)
    
    # 첫번재 인덱스, 인덱스의 리스트, 마지막 인덱스를 조합하여 호차별로 운행기록을 구분합니다.
    index = 0
    tupleList = []
    for i in indexList:
        tupleList.append((index, i))
        display(sameBusList.iloc[index:i].groupby(by=['stationSeq']).min())
        index=i
        
    print(plateNo, tupleList)
    break

(50.0, 11.0)
(53.0, 16.0)
[129, 240, 331]


,DateTime,endBus,lowPlate,plateNo,plateType,remainSeatCnt,routeId,stationId
stationSeq,,,,,,,,
6.0,2020-11-13T16:07:43.743358+09:00,0,0,경기70아7361,3,44,204000046,206000006
7.0,2020-11-13T16:09:35.156841+09:00,0,0,경기70아7361,3,39,204000046,206000005
8.0,2020-11-13T16:10:30.930773+09:00,0,0,경기70아7361,3,39,204000046,206000004
9.0,2020-11-13T16:13:17.798959+09:00,0,0,경기70아7361,3,39,204000046,206000555
10.0,2020-11-13T16:14:13.444355+09:00,0,0,경기70아7361,3,38,204000046,206000003
11.0,2020-11-13T16:16:59.818394+09:00,0,0,경기70아7361,3,36,204000046,206000002
12.0,2020-11-13T16:22:38.829872+09:00,0,0,경기70아7361,3,36,204000046,277103149
13.0,2020-11-13T16:25:25.373548+09:00,0,0,경기70아7361,3,36,204000046,277103679
14.0,2020-11-13T16:27:23.102140+09:00,0,0,경기70아7361,3,36,204000046,277103441


,DateTime,endBus,lowPlate,plateNo,plateType,remainSeatCnt,routeId,stationId
stationSeq,,,,,,,,
11.0,2020-11-13T19:02:24.539285+09:00,0,0,경기70아7361,3,39,204000046,206000002
12.0,2020-11-13T19:05:12.184647+09:00,0,0,경기70아7361,3,39,204000046,277103149
13.0,2020-11-13T19:07:05.568401+09:00,0,0,경기70아7361,3,39,204000046,277103679
14.0,2020-11-13T19:09:01.459581+09:00,0,0,경기70아7361,3,39,204000046,277103441
15.0,2020-11-13T19:11:49.504569+09:00,0,0,경기70아7361,3,39,204000046,277104177
16.0,2020-11-13T19:14:36.919469+09:00,0,0,경기70아7361,3,39,204000046,277103388
17.0,2020-11-13T19:18:21.848695+09:00,0,0,경기70아7361,3,39,204000046,277103390
18.0,2020-11-13T19:20:13.818687+09:00,0,0,경기70아7361,3,39,204000046,277103391
19.0,2020-11-13T19:23:07.105679+09:00,0,0,경기70아7361,3,39,204000046,277103400


,DateTime,endBus,lowPlate,plateNo,plateType,remainSeatCnt,routeId,stationId
stationSeq,,,,,,,,
16.0,2020-11-13T22:23:07.047239+09:00,0,0,경기70아7361,3,40,204000046,277103388
17.0,2020-11-13T22:26:01.704220+09:00,0,0,경기70아7361,3,40,204000046,277103390
18.0,2020-11-13T22:29:47.195579+09:00,0,0,경기70아7361,3,40,204000046,277103391
19.0,2020-11-13T22:34:24.462490+09:00,0,0,경기70아7361,3,40,204000046,277103400
20.0,2020-11-13T22:38:06.321017+09:00,0,0,경기70아7361,3,40,204000046,102000069
21.0,2020-11-13T22:39:01.742333+09:00,0,0,경기70아7361,3,40,204000046,277104180
22.0,2020-11-13T22:40:52.582987+09:00,0,0,경기70아7361,3,40,204000046,277104186
23.0,2020-11-13T22:43:39.347566+09:00,0,0,경기70아7361,3,40,204000046,101000292
24.0,2020-11-13T22:46:28.785177+09:00,0,0,경기70아7361,3,42,204000046,100000001


경기70아7361 [(0, 129), (129, 240), (240, 331)]


In [28]:
# 각 차량 번호를 기준으로 반복문을 수행합니다.
for plateNo in plateNoList:
    
    # 특정 차량번호와 같은 운행기록을 "DateTime을 기준으로 오름차순으로 정렬합니다."
    sameBusList = _df[_df['plateNo'] == plateNo].sort_values(['DateTime']).reset_index(drop=True)
    
    # 버스는 뒤로가지 않으므로
    # "stationSeq"가 작아지는 인덱스 i의 리스트를 만듭니다.
    indexList = []
    for i in range(1,len(sameBusList)):
        if(sameBusList.iloc[i-1]['stationSeq'] > sameBusList.iloc[i]['stationSeq']):
            indexList.append(i)
    indexList.append(len(sameBusList))
    
    # 첫번재 인덱스, 인덱스의 리스트, 마지막 인덱스를 조합하여 호차별로 운행기록을 구분합니다.
    index = 0
    tupleList = []
    for i in indexList:
        tupleList.append((index, i))
        index=i
    print(plateNo, tupleList)

경기70아7474 [(0, 8), (8, 89), (89, 100), (100, 197), (197, 208), (208, 303), (303, 313), (313, 405)]
경기70아8601 [(0, 9), (9, 93), (93, 102), (102, 193), (193, 205), (205, 313), (313, 325), (325, 418), (418, 430)]
경기70아8629 [(0, 11), (11, 107), (107, 115), (115, 205), (205, 216), (216, 316), (316, 326), (326, 410), (410, 418), (418, 450)]
경기70아8789 [(0, 8), (8, 94), (94, 102), (102, 191), (191, 200), (200, 290), (290, 299), (299, 396), (396, 405), (405, 480)]
경기70아8880 [(0, 8), (8, 94), (94, 105), (105, 188), (188, 200), (200, 294), (294, 306), (306, 395), (395, 405), (405, 457)]
경기77바1097 [(0, 9), (9, 87), (87, 96), (96, 196), (196, 208), (208, 311), (311, 318), (318, 415)]
경기78아1147 [(0, 13), (13, 117), (117, 132), (132, 248), (248, 264), (264, 374), (374, 387), (387, 477), (477, 522)]
경기78아1178 [(0, 8), (8, 95), (95, 106), (106, 194), (194, 204), (204, 302), (302, 311), (311, 394), (394, 403), (403, 412)]


In [73]:
sameBusList

,DateTime,endBus,lowPlate,plateNo,plateType,remainSeatCnt,routeId,stationId,stationSeq
0,2020-11-13T16:32:29.570798+09:00,0,0,경기77바1350,3,45,234000065,277102412,2.0
1,2020-11-13T16:33:25.559055+09:00,0,0,경기77바1350,3,44,234000065,228001428,3.0
2,2020-11-13T16:34:21.809741+09:00,0,0,경기77바1350,3,44,234000065,228001428,3.0
3,2020-11-13T16:35:16.946563+09:00,0,0,경기77바1350,3,44,234000065,228000346,4.0
4,2020-11-13T16:36:12.186446+09:00,0,0,경기77바1350,3,44,234000065,228000345,5.0
5,2020-11-13T16:37:07.330492+09:00,0,0,경기77바1350,3,44,234000065,228000299,6.0
6,2020-11-13T16:38:02.578296+09:00,0,0,경기77바1350,3,44,234000065,228000298,7.0
7,2020-11-13T16:38:58.123781+09:00,0,0,경기77바1350,3,44,234000065,228000297,8.0
8,2020-11-13T16:39:55.545111+09:00,0,0,경기77바1350,3,44,234000065,228000296,9.0
9,2020-11-13T16:40:52.747583+09:00,0,0,경기77바1350,3,44,234000065,228000296,9.0


In [32]:
# _df.groupby(['plateNo']).size()
_df = _df.sort_values(['plateNo'])
_df = _df[_df['plateNo'] == '경기70아7361']
_df
# _df = _df.sort_values(['stationSeq'])
_df.groupby(['stationSeq']).min()

,DateTime,endBus,lowPlate,plateNo,plateType,remainSeatCnt,routeId,stationId
stationSeq,,,,,,,,
6,2020-11-13T16:07:43.743358+09:00,0,0,경기70아7361,3,44,204000046,206000006
7,2020-11-13T16:09:35.156841+09:00,0,0,경기70아7361,3,39,204000046,206000005
8,2020-11-13T16:10:30.930773+09:00,0,0,경기70아7361,3,39,204000046,206000004
9,2020-11-13T16:13:17.798959+09:00,0,0,경기70아7361,3,39,204000046,206000555
10,2020-11-13T16:14:13.444355+09:00,0,0,경기70아7361,3,38,204000046,206000003
11,2020-11-13T16:16:59.818394+09:00,0,0,경기70아7361,3,36,204000046,206000002
12,2020-11-13T16:22:38.829872+09:00,0,0,경기70아7361,3,36,204000046,277103149
13,2020-11-13T16:25:25.373548+09:00,0,0,경기70아7361,3,36,204000046,277103679
14,2020-11-13T16:27:23.102140+09:00,0,0,경기70아7361,3,36,204000046,277103441
